# Matrix Product States

In [1]:
using Tenet

In this example, we will be using the `MatrixProductState` or `MPS` type. In order to construct one, you can pass the arrays.

In [2]:
ψ = MPS([rand(ComplexF64, 2,2), rand(ComplexF64, 2, 2, 2), rand(ComplexF64, 2, 2)])

MatrixProductState (#tensors=3, #inds=5)

`CanonicalForm` or `form` returns the canonical form trait. Note that since we added the arrays directly, and there is no information about the orthogonality center, `MixedCanonicalMPS` interprets it as if the orthogonality center spans all the sites.

In [3]:
form(ψ)

MixedCanonical{Vector{Site}}(Site[(2,), (1,), (3,)])

In order to canonize it, you may call the `canonize!` function together with the `CanonicalForm` trait, which in this case should be `MixedCanonical(site_where_to_canonize)`.

In [4]:
canonize!(ψ, MixedCanonical(site"1"))

MatrixProductState (#tensors=3, #inds=5)

In [5]:
form(ψ)

MixedCanonical{CartesianSite{1}}((1,))

You can directly call `norm` and `normalize!` methods from `LinearAlgebra` too.

In [6]:
norm(ψ)

4.312455962071073

In [7]:
normalize!(ψ)

MatrixProductState (#tensors=3, #inds=5)

In [8]:
norm(ψ)

0.9999999999999999

Note that these methods may canonize or move the orthogonality center.

In [9]:
canonize!(ψ, MixedCanonical(all_sites(ψ)))
form(ψ)

MixedCanonical{Vector{Site}}(Site[(2,), (1,), (3,)])

In [10]:
norm(ψ)
form(ψ)

MixedCanonical{CartesianSite{1}}((1,))

## Time evolution

### Gate application using the "Simple Update" routine

In order to create an operator, you may use a regular `Tensor` but where the indices are `Index{Plug}`. This way, Tenet knows which `Tensor` indices connect with the `Tensor`s of the MPS.

In [11]:
op = Tensor(rand(ComplexF64, 2, 2, 2, 2), [Index(plug"2"), Index(plug"3"), Index(plug"2'"), Index(plug"3'")])

2×2×2×2 Tensor{ComplexF64, 4, Array{ComplexF64, 4}}:
[:, :, 1, 1] =
 0.913872+0.805562im  0.329382+0.934858im
 0.688747+0.642765im  0.694356+0.662784im

[:, :, 2, 1] =
 0.415378+0.915594im  0.283779+0.0548586im
 0.299637+0.447855im  0.207589+0.72196im

[:, :, 1, 2] =
  0.38512+0.441854im  0.489417+0.884673im
 0.467067+0.453583im  0.967792+0.861111im

[:, :, 2, 2] =
 0.641606+0.281474im  0.729744+0.542118im
 0.593045+0.633061im  0.849462+0.176387im

In order to apply the operator, you can call the `simple_update!` function.

In [12]:
simple_update!(ψ, op)

MatrixProductState (#tensors=3, #inds=5)

As with the `norm` and `normalize!` functions, it canonizes to the acting sites of the gate for numerical precision.

In [13]:
form(ψ)

MixedCanonical{Vector{CartesianSite{1}}}(CartesianSite{1}[(2,), (3,)])

### MPS-MPO contraction

In order to construct a `MatrixProductOperator` or `MPO`, you can directly pass the arrays or construct one random with `rand`.

In [14]:
mpo = rand(MPO; n=3, maxdim=4)

MatrixProductOperator (#tensors=3, #inds=8)

In order to contract the MPS and the MPO, use the general purpose `evolve!` function.

In [15]:
evolve!(ψ, mpo)

MatrixProductState (#tensors=3, #inds=5)

Note that, currently, the bond dimension of the MPS increases with the evolution.

In [16]:
for bond in all_bonds(ψ)
    println("bond = $bond --> size = $(size(ψ, ind_at(ψ, bond)))")
end

bond = (1,) <=> (2,) --> size = 8
bond = (2,) <=> (3,) --> size = 8


## Compression

In order to reduce the size of the virtual bonds, you may use the `compress!` function. It accepts `maxdim` and `threshold` (which truncates based on `abs`olute value of the singular values) kwargs.

In [17]:
compress!(ψ; maxdim=4)

ArgumentError: ArgumentError: `BackendBase` can't deal with hyperindices. Load OMEinsum and use `BackendOMEinsum` instead.
isdisjoint(inds_c, inds_contract) must hold. Got
inds_c => Index[index<(1,) <=> (2,)>, index<##tmp#261>]
inds_contract => Index[index<##tmp#261>]

In [18]:
for bond in all_bonds(ψ)
    println("bond = $bond --> size = $(size(ψ, ind_at(ψ, bond)))")
end

bond = (1,) <=> (2,) --> size = 4
bond = (2,) <=> (3,) --> size = 2
